# Arivale multi-omic interaction analysis

Inputs: 
* Preprocessed data: Arivale_preprocessed_interaction_analysis
* List of analytes: Arivale_Interaction_Analysis_Analyte_List.json
* Clinical chem annotations: Arivale_Interaction_Clinical_Annots.json
* Protein annotations: Arivale_Interaction_Prots_Annots.json
* Metabolite annotations: Arivale_Interaction_Metabs_Annots.json
* Microbiome annotations:  Arivale_Interaction_Microb_Annots.json

Outputs: log files and result data files to be used in Code03

'Save' lines are commented out  
'Healthy' and 'Unhealthy' are renamed as 'Bio_Young' and 'Bio_Old' in figures and data files where appropriate  

Note: script ran in current form will yield results for female APOE E2; commenting in/out specified lines will produce results for male E2; male/female APOE E4; biologically younger males/females; and biologically older males/females in the first section. The second section can be commented in/out in the same manner to produce results for the e4 allele dosage and continuous delta age interactions (currently commented in for e2 allele dosage). Copies of the script can thus be run in parallel to avoid time bottleneck of running 11 lengthy analysis in series.

In [ ]:
# Load packages
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy
import sys
import math
from datetime import datetime
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('analytics')
logger.setLevel(logging.INFO)
from analytics.util.analytics_logger import GetAnalyticsLogger
import logging
logger = GetAnalyticsLogger()
logger.setLevel(logging.INFO)
from importlib.metadata import version
import json

# Load in Data, Analyte Lists/Metadata

In [ ]:
df_with_apoe = pd.read_csv('/notebooks/0. APOE-Multiomics/Data_Files/Arivale_preprocessed_interaction_analysis.csv')

In [ ]:
with open("/notebooks/0. APOE-Multiomics/Data_Files/Arivale_Interaction_Analysis_Analyte_List.json", "r") as file:
    analytes = json.load(file)

In [ ]:
with open("/notebooks/0. APOE-Multiomics/Data_Files/Arivale_Interaction_Clinical_Annots.json", "r") as file:
    clinical = json.load(file)

In [ ]:
with open("/notebooks/0. APOE-Multiomics/Data_Files/Arivale_Interaction_Prots_Annots.json", "r") as file:
    proteins = json.load(file)

In [ ]:
with open("/notebooks/0. APOE-Multiomics/Data_Files/Arivale_Interaction_Metabs_Annots.json", "r") as file:
    metabolites = json.load(file)

In [ ]:
with open("/notebooks/0. APOE-Multiomics/Data_Files/Arivale_Interaction_Microb_Annots.json", "r") as file:
    microbiome = json.load(file)

# Get pairs for GLM

In [ ]:
import statsmodels

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.sandbox.stats.multicomp
from statsmodels.genmod.families import family, links
import itertools

In [ ]:
pairs = list(itertools.combinations(analytes, 2))

In [ ]:
# generate all pairs to be tested
def count_pairs_interomic(analytes, proteins, metabolites, clinical, microbiome):

    # Create all possible pairs of inputs
    pairs = list(itertools.combinations(analytes, 2))
#     dfpairs = pd.DataFrame(pairs)
#     dfpairs.rename(columns = {0:'col1',1:'col2'}, inplace = True)
    logger.info('Created {} pairs'.format(len(pairs))) # repeat this for ____ inter-omic pairs, then finally print those that passed
    
    interomic_pairs = list()
    
    count_big = 0
    count = 0
    
    start_time = datetime.now()
    
    for (col1, col2) in pairs:
        count_big += 1
        if (count_big % 100000) == 0:
            elapsed_time = datetime.now() - start_time    
            logger.info('Started number {} at {:.3f} seconds'.format(count_big, elapsed_time.total_seconds()))

        if (col1 in metabolites.keys()) and (col2 in metabolites.keys()):
            continue

        if (col1 in proteins.keys()) and (col2 in proteins.keys()):
            continue

        if (col1 in clinical.keys()) and (col2 in clinical.keys()):
            continue
            
        if (col1 in microbiome.keys()) and (col2 in microbiome.keys()):
            continue
            
        else:
            interomic_pairs.append((col1,col2))
            
        count += 1
        
        
    logger.info('Identified {} interomic pairs'.format(len(interomic_pairs)))
        
    print('Running {} pairs'.format(count))
    
    return interomic_pairs

In [ ]:
interomic_pairs = count_pairs_interomic(analytes, proteins, metabolites, clinical, microbiome)

# Function for interaction analysis - to be edited for each subset to test

In [ ]:
# comment in/out lines to analyze each group individually (female or male for E2, E4, bio young, bio old)
def run_screened_glm_pairwise_E2_compare(screened_pairs, dat, analytes, proteins, metabolites, clinical, microbiome, max_run=None): # for E2
# def run_screened_glm_pairwise_E4_compare(screened_pairs, dat, analytes, proteins, metabolites, clinical, microbiome, max_run=None): # for E4
# def run_screened_glm_pairwise_health_compare(screened_pairs, dat, analytes, proteins, metabolites, clinical, microbiome, max_run=None): # for bio young
# def run_screened_glm_pairwise_unhealth_compare(screened_pairs, dat, analytes, proteins, metabolites, clinical, microbiome, max_run=None): # for bio old

    print('Running {} pairs'.format(len(screened_pairs)))
         
    count = 0 
    done = 0
    skipped = 0
    completed = 0

    results = []
#     columns = list(dat.columns)

    start_time = datetime.now()
    for (col1, col2) in screened_pairs:
            
        if (col1 in proteins.keys()):
            name1 = proteins[col1]['name']
            subgroup1 = proteins[col1]['sub_group'] 
            supergroup1 = proteins[col1]['super_group']
            type1 = 'protein'
            mark1 = 'has_protein'
        elif (col1 in metabolites.keys()):
            name1 = metabolites[col1]['name']
            subgroup1 = metabolites[col1]['sub_group'] 
            supergroup1 = metabolites[col1]['super_group']
            type1 = 'metabolite'
            mark1 = 'has_metab'
        elif (col1 in clinical.keys()):
            name1 = clinical[col1]['name']
            subgroup1 = clinical[col1]['sub_group'] 
            supergroup1 = clinical[col1]['super_group']
            type1 = 'clinical'
            mark1 = 'has_clinical'
        elif (col1 in microbiome.keys()):
            name1 = microbiome[col1]['name']
            subgroup1 = microbiome[col1]['sub_group'] 
            supergroup1 = microbiome[col1]['super_group']
            mark1 = 'has_microbio'
        else:
            print('Col1 {} not found in any annotation'.format(col1))
            name1 = None
            subgroup1 = None
            supergroup1 = None
            type1 = None
            mark1 = None
            
        if (col2 in proteins.keys()):
            name2 = proteins[col2]['name']
            subgroup2 = proteins[col2]['sub_group'] 
            supergroup2 = proteins[col2]['super_group']
            type2 = 'protein'
            mark2 = 'has_protein'
        elif (col2 in metabolites.keys()):
            name2 = metabolites[col2]['name']
            subgroup2 = metabolites[col2]['sub_group'] 
            supergroup2 = metabolites[col2]['super_group']
            type2 = 'metabolite'
            mark2 = 'has_metab'
        elif (col2 in clinical.keys()):
            name2 = clinical[col2]['name']
            subgroup2 = clinical[col2]['sub_group'] 
            supergroup2 = clinical[col2]['super_group']
            type2 = 'clinical'
            mark2 = 'has_clinical'
        elif (col2 in microbiome.keys()):
            name2 = microbiome[col2]['name']
            subgroup2 = microbiome[col2]['sub_group'] 
            supergroup2 = microbiome[col2]['super_group']
            type2 = 'microbiome'
            mark2 = 'has_microbio'
        else:
            print('Col2 {} not found in any annotation'.format(col2))
            name2 = None
            subgroup2 = None
            supergroup2 = None
            type2 = None
            mark2 = None

        # Default is gaussian
        family_type = family.Gaussian()
        family_type.link = links.identity()
        family_name = 'Gaussian'
        family_link = 'Identity'

        # Covariance structure
        cov = sm.cov_struct.Exchangeable()

        sub = dat[['public_client_id', col1, col2, 'age', 'APOE_Status', 'Model_Health', 'season', 'BMI_CALC', 'meds_cholesterol', 'PC1', 'PC2', mark1, mark2]].copy() # 'meds_blood_sugar', 'meds_blood_pressure', 'PC3', 'PC4'
        sub.dropna(subset = [mark1,mark2], inplace=True)
        sub.rename(columns={col1:'analyte1'}, inplace=True)
        sub.rename(columns={col2:'analyte2'}, inplace=True)

        if (sub['analyte1'].skew() > 1.5) | (sub['analyte1'].skew() < -1.5):

            #logger.info('Setting gamma family for skewed analyte %s'%(col))

            # Set any zero values to 1/2 the smallest value
            sub.loc[sub['analyte1']==0, 'analyte1'] = (sub.loc[sub['analyte1']>0, 'analyte1'].min() / 2.0)

            family_type = family.Gamma()
            family_type.link = links.log()
            family_name = 'Gamma'
            family_link = 'Log'

        try:

#             ols_model = 'analyte1 ~ age*analyte2*C(sex) + C(season) + BMI_CALC + C(meds_cholesterol) + C(meds_blood_sugar) + C(meds_blood_pressure) + C(vendor) + PC1 + PC2 + PC3 + PC4'

### comment the ols models below in/out to select analysis

            #E2:
            ols_model = 'analyte1 ~ analyte2*C(APOE_Status, Treatment(reference=1)) + age + C(season) + BMI_CALC + C(meds_cholesterol) + PC1 + PC2'  # C(meds_blood_sugar) + C(meds_blood_pressure) + PC3 + PC4 + C(vendor)
        
            #E4:
            # ols_model = 'analyte1 ~ analyte2*C(APOE_Status) + age + C(season) + BMI_CALC + C(meds_cholesterol) + PC1 + PC2'  # C(meds_blood_sugar) + C(meds_blood_pressure) + PC3 + PC4 + C(vendor)
            
            #Bio_Young:
            # ols_model = 'analyte1 ~ analyte2*C(Model_Health, Treatment(reference=1)) + age + C(season) + BMI_CALC + C(meds_cholesterol) + PC1 + PC2'  # C(meds_blood_sugar) + C(meds_blood_pressure) + PC3 + PC4 + C(vendor)
            
            #Bio_Old:
            # ols_model = 'analyte1 ~ analyte2*C(Model_Health) + age + C(season) + BMI_CALC + C(meds_cholesterol) + PC1 + PC2'  # C(meds_blood_sugar) + C(meds_blood_pressure) + PC3 + PC4 + C(vendor)
        
            fitted_model = smf.glm(ols_model, data=sub, family=family_type, missing='drop').fit(maxiter=2000)
            results.append((col1, name1, type1, supergroup1, subgroup1, col2, name2, type2, supergroup2, subgroup2, len(fitted_model.fittedvalues), fitted_model.converged, *fitted_model.params, *fitted_model.pvalues))

        except Exception as e:
            print('Failed analytes {} {} with error {}'.format(col1, col2, str(e)))
            skipped += 1

        count += 1
        if (max_run is not None) and (count >= max_run):
            break
        
        if (count % 1000) == 0:

            elapsed_time = datetime.now() - start_time    
            print('Finished {} in {:.3f} seconds (skipped {})'.format(count, elapsed_time.total_seconds(), skipped))
      
    elapsed_time = datetime.now() - start_time    
    print('Complete! Yay! Finished {} in {:.3f} seconds (skipped {})'.format(count, elapsed_time.total_seconds(), skipped))


### comment sections below in/out to select analysis
    
    #E2:
    df = pd.DataFrame(results, columns=['col1', 'name1', 'type1', 'supergroup1', 'subgroup1', 'col2', 'name2', 'type2', 'supergroup2', 'subgroup2', 'n', 'converged', *fitted_model.params.index, *[str(x)+'_p' for x in fitted_model.pvalues.index]])
    df.sort_values(['analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p'], ascending=True, inplace=True)
    
    np.seterr(all='warn')
    (adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(df.loc[~df['analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p'].isnull(), 'analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p'], alpha=0.05, method='fdr_bh')
    df.loc[~df['analyte2:C(APOE_Status, Treatment(reference=1))[T.E2]_p'].isnull(), 'pval_adj'] = adj_pval
    df.sort_values(['pval_adj'], ascending=True, inplace=True)
    
    #E4:
#     df = pd.DataFrame(results, columns=['col1', 'name1', 'type1', 'supergroup1', 'subgroup1', 'col2', 'name2', 'type2', 'supergroup2', 'subgroup2', 'n', 'converged', *fitted_model.params.index, *[str(x)+'_p' for x in fitted_model.pvalues.index]])
#     df.sort_values(['analyte2:C(APOE_Status)[T.E4]_p'], ascending=True, inplace=True)
    
#     np.seterr(all='warn')
#     (adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(df.loc[~df['analyte2:C(APOE_Status)[T.E4]_p'].isnull(), 'analyte2:C(APOE_Status)[T.E4]_p'], alpha=0.05, method='fdr_bh')
#     df.loc[~df['analyte2:C(APOE_Status)[T.E4]_p'].isnull(), 'pval_adj'] = adj_pval
#     df.sort_values(['pval_adj'], ascending=True, inplace=True)
            
    #Bio_Young:
#     df = pd.DataFrame(results, columns=['col1', 'name1', 'type1', 'supergroup1', 'subgroup1', 'col2', 'name2', 'type2', 'supergroup2', 'subgroup2', 'n', 'converged', *fitted_model.params.index, *[str(x)+'_p' for x in fitted_model.pvalues.index]])
#     df.sort_values(['analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]_p'], ascending=True, inplace=True)
    
#     np.seterr(all='warn')
#     (adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(df.loc[~df['analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]_p'].isnull(), 'analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]_p'], alpha=0.05, method='fdr_bh')
#     df.loc[~df['analyte2:C(Model_Health, Treatment(reference=1))[T.Healthy]_p'].isnull(), 'pval_adj'] = adj_pval
#     df.sort_values(['pval_adj'], ascending=True, inplace=True)
            
    #Bio_Old:
#     df = pd.DataFrame(results, columns=['col1', 'name1', 'type1', 'supergroup1', 'subgroup1', 'col2', 'name2', 'type2', 'supergroup2', 'subgroup2', 'n', 'converged', *fitted_model.params.index, *[str(x)+'_p' for x in fitted_model.pvalues.index]])
#     df.sort_values(['analyte2:C(Model_Health)[T.Unhealthy]_p'], ascending=True, inplace=True)
    
#     np.seterr(all='warn')
#     (adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(df.loc[~df['analyte2:C(Model_Health)[T.Unhealthy]_p'].isnull(), 'analyte2:C(Model_Health)[T.Unhealthy]_p'], alpha=0.05, method='fdr_bh')
#     df.loc[~df['analyte2:C(Model_Health)[T.Unhealthy]_p'].isnull(), 'pval_adj'] = adj_pval
#     df.sort_values(['pval_adj'], ascending=True, inplace=True)
    
    return df

In [ ]:
# restrict data to subset to be tested
df_F = df_with_apoe[(df_with_apoe.sex == 'F')]
# df_M = df_with_apoe[(df_with_apoe.sex == 'M')]

df_FE2 = df_F[(df_F.APOE_Status == 'E3')|(df_F.APOE_Status == 'E2')]
# df_FE4 = df_F[(df_F.APOE_Status == 'E3')|(df_F.APOE_Status == 'E4')]
# df_Fhealth = df_F[(df_F.Model_Health == 'Norm')|(df_F.Model_Health == 'Healthy')]
# df_Funhealth = df_F[(df_F.Model_Health == 'Norm')|(df_F.Model_Health == 'Unhealthy')]

# df_ME2 = df_M[(df_F.APOE_Status == 'E3')|(df_M.APOE_Status == 'E2')]
# df_ME4 = df_M[(df_F.APOE_Status == 'E3')|(df_M.APOE_Status == 'E4')]
# df_Mhealth = df_M[(df_F.Model_Health == 'Norm')|(df_M.Model_Health == 'Healthy')]
# df_Munhealth = df_M[(df_F.Model_Health == 'Norm')|(df_M.Model_Health == 'Unhealthy')]

# Test on small data set to ensure working before beginning long analysis

In [ ]:
import random

In [ ]:
test_pairs = random.sample(interomic_pairs, 16)

In [ ]:
import sys
sys.stdout = open("TEST_5SD_Arivale_sep.txt", "a")

In [ ]:
print("testing")

In [ ]:
temp = run_screened_glm_pairwise_E2_compare(test_pairs, df_FE2, analytes, proteins, metabolites, clinical, microbiome)
# temp = run_screened_glm_pairwise_E4_compare(test_pairs, df_FE4, analytes, proteins, metabolites, clinical, microbiome)
# temp = run_screened_glm_pairwise_health_compare(test_pairs, df_Fhealth, analytes, proteins, metabolites, clinical, microbiome)
# temp = run_screened_glm_pairwise_unhealth_compare(test_pairs, df_Funhealth, analytes, proteins, metabolites, clinical, microbiome)

# temp = run_screened_glm_pairwise_E2_compare(test_pairs, df_ME2, analytes, proteins, metabolites, clinical, microbiome)
# temp = run_screened_glm_pairwise_E4_compare(test_pairs, df_ME4, analytes, proteins, metabolites, clinical, microbiome)
# temp = run_screened_glm_pairwise_health_compare(test_pairs, df_Mhealth, analytes, proteins, metabolites, clinical, microbiome)
# temp = run_screened_glm_pairwise_unhealth_compare(test_pairs, df_Munhealth, analytes, proteins, metabolites, clinical, microbiome)

In [ ]:
temp # ensure data presence

In [ ]:
temp.columns.to_list() # ensure desired columns

# Run Analysis

In [ ]:
import sys
sys.stdout = open("240427_FE2_interact_log_Arivale_5SD_winsor.txt", "a")
# sys.stdout = open("240427_FE4_interact_log_Arivale_5SD_winsor.txt", "a")
# sys.stdout = open("240427_Fhealth_interact_log_Arivale_5SD_winsor.txt", "a")
# sys.stdout = open("240427_Funhealth_interact_log_Arivale_5SD_winsor.txt", "a")

# sys.stdout = open("240427_ME2_interact_log_Arivale_5SD_winsor.txt", "a")
# sys.stdout = open("240427_ME4_interact_log_Arivale_5SD_winsor.txt", "a")
# sys.stdout = open("240427_Mhealth_interact_log_Arivale_5SD_winsor.txt", "a")
# sys.stdout = open("240427_Munhealth_interact_log_Arivale_5SD_winsor.txt", "a")

In [ ]:
interact_glm_FE2_compare = run_screened_glm_pairwise_E2_compare(interomic_pairs, df_FE2, analytes, proteins, metabolites, clinical, microbiome)
# interact_glm_FE4_compare = run_screened_glm_pairwise_E4_compare(interomic_pairs, df_FE4, analytes, proteins, metabolites, clinical, microbiome)
# interact_glm_Fhealth_compare = run_screened_glm_pairwise_health_compare(interomic_pairs, df_Fhealth, analytes, proteins, metabolites, clinical, microbiome)
# interact_glm_Funhealth_compare = run_screened_glm_pairwise_unhealth_compare(interomic_pairs, df_Funhealth, analytes, proteins, metabolites, clinical, microbiome)

# interact_glm_ME2_compare = run_screened_glm_pairwise_E2_compare(interomic_pairs, df_ME2, analytes, proteins, metabolites, clinical, microbiome)
# interact_glm_ME4_compare = run_screened_glm_pairwise_E4_compare(interomic_pairs, df_ME4, analytes, proteins, metabolites, clinical, microbiome)
# interact_glm_Mhealth_compare = run_screened_glm_pairwise_health_compare(interomic_pairs, df_Mhealth, analytes, proteins, metabolites, clinical, microbiome)
# interact_glm_Munhealth_compare = run_screened_glm_pairwise_unhealth_compare(interomic_pairs, df_Munhealth, analytes, proteins, metabolites, clinical, microbiome)

In [ ]:
interact_glm_FE2_compare.to_csv('240427_FE2_interomic_interact_Arivale_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')
# interact_glm_FE4_compare.to_csv('240427_FE4_interomic_interact_Arivale_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')
# interact_glm_Fhealth_compare.to_csv('240427_Fhealth_interomic_interact_Arivale_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')
# interact_glm_Funhealth_compare.to_csv('240427_Funhealth_interomic_interact_Arivale_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')

# interact_glm_ME2_compare.to_csv('240427_ME2_interomic_interact_Arivale_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')
# interact_glm_ME4_compare.to_csv('240427_ME4_interomic_interact_Arivale_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')
# interact_glm_Mhealth_compare.to_csv('240427_Mhealth_interomic_interact_Arivale_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')
# interact_glm_Munhealth_compare.to_csv('240427_Munhealth_interomic_interact_Arivale_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')

---

# Function for interaction analysis of e2 or e4 allele dosage or continuous delta age - to be edited for each subset to test

In [ ]:
# comment in/out lines to analyze each group individually (e2 alleles, e4 alleles, or delta age)
def run_screened_glm_pairwise_e2_allele_compare(screened_pairs, dat, analytes, proteins, metabolites, clinical, microbiome, max_run=None): # for e2 allele
# def run_screened_glm_pairwise_e4_allele_compare(screened_pairs, dat, analytes, proteins, metabolites, clinical, microbiome, max_run=None): # for e4 allele
# def run_screened_glm_pairwise_delage_compare(screened_pairs, dat, analytes, proteins, metabolites, clinical, microbiome, max_run=None): # for delta age, continuous

    print('Running {} pairs'.format(len(screened_pairs)))
         
    count = 0 
    done = 0
    skipped = 0
    completed = 0

    results = []
#     columns = list(dat.columns)

    start_time = datetime.now()
    for (col1, col2) in screened_pairs:
            
        if (col1 in proteins.keys()):
            name1 = proteins[col1]['name']
            subgroup1 = proteins[col1]['sub_group'] 
            supergroup1 = proteins[col1]['super_group']
            type1 = 'protein'
            mark1 = 'has_protein'
        elif (col1 in metabolites.keys()):
            name1 = metabolites[col1]['name']
            subgroup1 = metabolites[col1]['sub_group'] 
            supergroup1 = metabolites[col1]['super_group']
            type1 = 'metabolite'
            mark1 = 'has_metab'
        elif (col1 in clinical.keys()):
            name1 = clinical[col1]['name']
            subgroup1 = clinical[col1]['sub_group'] 
            supergroup1 = clinical[col1]['super_group']
            type1 = 'clinical'
            mark1 = 'has_clinical'
        elif (col1 in microbiome.keys()):
            name1 = microbiome[col1]['name']
            subgroup1 = microbiome[col1]['sub_group'] 
            supergroup1 = microbiome[col1]['super_group']
            mark1 = 'has_microbio'
        else:
            print('Col1 {} not found in any annotation'.format(col1))
            name1 = None
            subgroup1 = None
            supergroup1 = None
            type1 = None
            mark1 = None
            
        if (col2 in proteins.keys()):
            name2 = proteins[col2]['name']
            subgroup2 = proteins[col2]['sub_group'] 
            supergroup2 = proteins[col2]['super_group']
            type2 = 'protein'
            mark2 = 'has_protein'
        elif (col2 in metabolites.keys()):
            name2 = metabolites[col2]['name']
            subgroup2 = metabolites[col2]['sub_group'] 
            supergroup2 = metabolites[col2]['super_group']
            type2 = 'metabolite'
            mark2 = 'has_metab'
        elif (col2 in clinical.keys()):
            name2 = clinical[col2]['name']
            subgroup2 = clinical[col2]['sub_group'] 
            supergroup2 = clinical[col2]['super_group']
            type2 = 'clinical'
            mark2 = 'has_clinical'
        elif (col2 in microbiome.keys()):
            name2 = microbiome[col2]['name']
            subgroup2 = microbiome[col2]['sub_group'] 
            supergroup2 = microbiome[col2]['super_group']
            type2 = 'microbiome'
            mark2 = 'has_microbio'
        else:
            print('Col2 {} not found in any annotation'.format(col2))
            name2 = None
            subgroup2 = None
            supergroup2 = None
            type2 = None
            mark2 = None

        # Default is gaussian
        family_type = family.Gaussian()
        family_type.link = links.identity()
        family_name = 'Gaussian'
        family_link = 'Identity'

        # Covariance structure
        cov = sm.cov_struct.Exchangeable()

        sub = dat[['public_client_id', col1, col2, 'age', 'e2_allele', 'e4_allele', 'comb_del_age', 'season', 'BMI_CALC', 'meds_cholesterol', 'PC1', 'PC2', mark1, mark2]].copy() # 'meds_blood_sugar', 'meds_blood_pressure', 'PC3', 'PC4'
        sub.dropna(subset = [mark1,mark2], inplace=True)
        sub.rename(columns={col1:'analyte1'}, inplace=True)
        sub.rename(columns={col2:'analyte2'}, inplace=True)

        if (sub['analyte1'].skew() > 1.5) | (sub['analyte1'].skew() < -1.5):

            #logger.info('Setting gamma family for skewed analyte %s'%(col))

            # Set any zero values to 1/2 the smallest value
            sub.loc[sub['analyte1']==0, 'analyte1'] = (sub.loc[sub['analyte1']>0, 'analyte1'].min() / 2.0)

            family_type = family.Gamma()
            family_type.link = links.log()
            family_name = 'Gamma'
            family_link = 'Log'

        try:

#             ols_model = 'analyte1 ~ age*analyte2*C(sex) + C(season) + BMI_CALC + C(meds_cholesterol) + C(meds_blood_sugar) + C(meds_blood_pressure) + C(vendor) + PC1 + PC2 + PC3 + PC4'

### comment the ols models below in/out to select analysis

            #e2 allele:
            ols_model = 'analyte1 ~ analyte2*e2_allele + age + C(season) + BMI_CALC + C(meds_cholesterol) + PC1 + PC2'  # C(meds_blood_sugar) + C(meds_blood_pressure) + PC3 + PC4 + C(vendor)
        
            #e4 allele:
            # ols_model = 'analyte1 ~ analyte2*e4_allele + age + C(season) + BMI_CALC + C(meds_cholesterol) + PC1 + PC2'  # C(meds_blood_sugar) + C(meds_blood_pressure) + PC3 + PC4 + C(vendor)
            
            #delta age:
            # ols_model = 'analyte1 ~ analyte2*comb_del_age + age + C(season) + BMI_CALC + C(meds_cholesterol) + PC1 + PC2'  # C(meds_blood_sugar) + C(meds_blood_pressure) + PC3 + PC4 + C(vendor)
            
            fitted_model = smf.glm(ols_model, data=sub, family=family_type, missing='drop').fit(maxiter=2000)
            results.append((col1, name1, type1, supergroup1, subgroup1, col2, name2, type2, supergroup2, subgroup2, len(fitted_model.fittedvalues), fitted_model.converged, *fitted_model.params, *fitted_model.pvalues))

        except Exception as e:
            print('Failed analytes {} {} with error {}'.format(col1, col2, str(e)))
            skipped += 1

        count += 1
        if (max_run is not None) and (count >= max_run):
            break
        
        if (count % 1000) == 0:

            elapsed_time = datetime.now() - start_time    
            print('Finished {} in {:.3f} seconds (skipped {})'.format(count, elapsed_time.total_seconds(), skipped))
      
    elapsed_time = datetime.now() - start_time    
    print('Complete! Yay! Finished {} in {:.3f} seconds (skipped {})'.format(count, elapsed_time.total_seconds(), skipped))
    
    
### comment the sections below in/out to select analysis
    
    #e2_allele:
    df = pd.DataFrame(results, columns=['col1', 'name1', 'type1', 'supergroup1', 'subgroup1', 'col2', 'name2', 'type2', 'supergroup2', 'subgroup2', 'n', 'converged', *fitted_model.params.index, *[str(x)+'_p' for x in fitted_model.pvalues.index]])
    df.sort_values(['analyte2:e2_allele_p'], ascending=True, inplace=True)
    
    np.seterr(all='warn')
    (adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(df.loc[~df['analyte2:e2_allele_p'].isnull(), 'analyte2:e2_allele_p'], alpha=0.05, method='fdr_bh')
    df.loc[~df['analyte2:e2_allele_p'].isnull(), 'pval_adj'] = adj_pval
    df.sort_values(['pval_adj'], ascending=True, inplace=True)

    #e4_allele:
#     df = pd.DataFrame(results, columns=['col1', 'name1', 'type1', 'supergroup1', 'subgroup1', 'col2', 'name2', 'type2', 'supergroup2', 'subgroup2', 'n', 'converged', *fitted_model.params.index, *[str(x)+'_p' for x in fitted_model.pvalues.index]])
#     df.sort_values(['analyte2:e4_allele_p'], ascending=True, inplace=True)
    
#     np.seterr(all='warn')
#     (adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(df.loc[~df['analyte2:e4_allele_p'].isnull(), 'analyte2:e4_allele_p'], alpha=0.05, method='fdr_bh')
#     df.loc[~df['analyte2:e4_allele_p'].isnull(), 'pval_adj'] = adj_pval
#     df.sort_values(['pval_adj'], ascending=True, inplace=True)
            
    #delta age:
#     df = pd.DataFrame(results, columns=['col1', 'name1', 'type1', 'supergroup1', 'subgroup1', 'col2', 'name2', 'type2', 'supergroup2', 'subgroup2', 'n', 'converged', *fitted_model.params.index, *[str(x)+'_p' for x in fitted_model.pvalues.index]])
#     df.sort_values(['analyte2:comb_del_age_p'], ascending=True, inplace=True)
    
#     np.seterr(all='warn')
#     (adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(df.loc[~df['analyte2:comb_del_age_p'].isnull(), 'analyte2:comb_del_age_p'], alpha=0.05, method='fdr_bh')
#     df.loc[~df['analyte2:comb_del_age_p'].isnull(), 'pval_adj'] = adj_pval
#     df.sort_values(['pval_adj'], ascending=True, inplace=True)
            

    
    return df



In [ ]:
df_with_apoe['e2_allele'] = 999
df_with_apoe['e4_allele'] = 999
for i in df_with_apoe.index:
    if df_with_apoe.APOE_Genotype[i] == 'E2/E2':
        df_with_apoe.e2_allele[i] = 2
        df_with_apoe.e4_allele[i] = 0
    if df_with_apoe.APOE_Genotype[i] == 'E2/E3':
        df_with_apoe.e2_allele[i] = 1
        df_with_apoe.e4_allele[i] = 0
    if df_with_apoe.APOE_Genotype[i] == 'E2/E4':
        df_with_apoe.e2_allele[i] = 1
        df_with_apoe.e4_allele[i] = 1
    if df_with_apoe.APOE_Genotype[i] == 'E3/E3':
        df_with_apoe.e2_allele[i] = 0
        df_with_apoe.e4_allele[i] = 0
    if df_with_apoe.APOE_Genotype[i] == 'E3/E4':
        df_with_apoe.e2_allele[i] = 0
        df_with_apoe.e4_allele[i] = 1
    if df_with_apoe.APOE_Genotype[i] == 'E4/E4':
        df_with_apoe.e2_allele[i] = 0
        df_with_apoe.e4_allele[i] = 2

In [ ]:
# restrict data to subset to be tested
df_alleles = df_with_apoe[(df_with_apoe['e2_allele'] < 3) | (df_with_apoe['e4_allele'] < 3)]
df_delage = df_with_apoe

# Test on small data set to ensure working before beginning long analysis

In [ ]:
import random

In [ ]:
test_pairs = random.sample(interomic_pairs, 16)

In [ ]:
import sys
sys.stdout = open("TEST_5SD_Arivale_sep.txt", "a")

In [ ]:
print("testing")

In [ ]:
temp = run_screened_glm_pairwise_e2_allele_compare(test_pairs, df_alleles, analytes, proteins, metabolites, clinical, microbiome)
# temp = run_screened_glm_pairwise_e4_allele_compare(test_pairs, df_alleles, analytes, proteins, metabolites, clinical, microbiome)
# temp = run_screened_glm_pairwise_delage_compare(test_pairs, df_delage, analytes, proteins, metabolites, clinical, microbiome)

In [ ]:
temp # ensure data presence

In [ ]:
temp.columns.to_list() # ensure desired columns

# Run Analysis

In [ ]:
import sys
sys.stdout = open("240504_E2_allele_dose_interact_log_5SD_winsor.txt", "a")
# sys.stdout = open("240504_E4_allele_dose_interact_log_5SD_winsor.txt", "a")
# sys.stdout = open("240504_cont_del_age_interact_log_5SD_winsor.txt", "a")

In [ ]:
interact_glm_e2_allele_dose_compare = run_screened_glm_pairwise_e2_allele_compare(interomic_pairs, df_alleles, analytes, proteins, metabolites, clinical, microbiome)
# interact_glm_e4_allele_dose_compare = run_screened_glm_pairwise_e4_allele_compare(interomic_pairs, df_alleles, analytes, proteins, metabolites, clinical, microbiome)
# interact_glm_delage_compare = run_screened_glm_pairwise_delage_compare(interomic_pairs, df_delage, analytes, proteins, metabolites, clinical, microbiome)

In [ ]:
interact_glm_e2_allele_dose_compare.to_csv('240504_E2_allele_dose_interomic_interaction_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')
# interact_glm_e4_allele_dose_compare.to_csv('240504_E4_allele_dose_interomic_interaction_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')
# interact_glm_delage_compare.to_csv('240504_contin_del_age_interomic_interaction_5SD_winsor.txt.gz', index=False, sep='\t', compression='gzip')